# For XML Extention



### Преобразование данных из реляционной модели в XML

Зачастую необходимо делать выгрузку из реляционной БД в формате XML для передачи их некоторому приложению или сервису. 

В SQL Server это можно выполнить с помощью инструкции FOR XML, добавляемой в конец оператора SELECT.

## Пример логической модели данных:

<img src="https://github.com/timoti1/T-SQL/blob/master/SQL/img/SwimmersDB.png?raw=1" />

### For XML Explicit

Первая попытка Microsoft предоставить инструмент генерации XML заданной структуры из реляционных данных.

Имеет ряд недостатков и сейчас практически не используется.

In [8]:
use tempdb
go

select  1             as Tag,
		NULL          as Parent,
		s.FirstName   as [Swimmer!1!FirstName],
		s.LastName    as [Swimmer!1!LastName],
		s.YearOfBirth as [Swimmer!1!YearOfBirth],
		NULL          as [Team!2!City], 
		NULL          as [Team!2!Name],
		NULL          as [Coach!3!FirstName],
		NULL          as [Coach!3!LastName]
from   dbo.Swimmer s
union ALL 
select  2             as Tag,
		1             as Parent,
		s.FirstName,
		s.LastName,
		s.YearOfBirth,
		sc.City, 
		sc.[Name],
		NULL,
		NULL
from dbo.Swimmer s
left join dbo.SwimmingClub sc   on sc.SwimmingClubID = s.SwimmingClubID
union ALL 
select  3             as Tag,
		1             as Parent,
		s.FirstName,
		s.LastName,
		s.YearOfBirth,
		NULL, 
		NULL,
		c.FirstName,
		c.LastName
from dbo.Swimmer s
left join dbo.SwimmerCoach xref on xref.SwimmerID = s.SwimmerID 
inner join dbo.Coach c          on c.CoachID = xref.CoachID
order by 3, 4, 5, 6, 7, 8, 9
for xml explicit



Commands completed successfully.

Total execution time: 00:00:00.0019871

(21 rows affected)

Total execution time: 00:00:00.0039637

XML_F52E2B61-18A1-11d1-B105-00805F49916B
"<Swimmer FirstName=""Gleb"" LastName=""Bondarec"" YearOfBirth=""2007""><Coach FirstName=""Алла"" LastName=""Усенок""/><Coach FirstName=""Виталий"" LastName=""Барташевич""/><Team City=""Минск"" Name=""ГЦОР Трактор""/></Swimmer><Swimmer FirstName=""Алексей"" LastName=""Рылько"" YearOfBirth=""2005""><Team City=""Minsk"" Name=""ДЮСШ Янтарь""/></Swimmer><Swimmer FirstName=""Анна"" LastName=""Высоцкая"" YearOfBirth=""2007""><Coach FirstName=""Евгения"" LastName=""Жукова""/><Team City=""Minsk"" Name=""ДЮСШ Янтарь""/></Swimmer><Swimmer FirstName=""Илья"" LastName=""Гавриленко"" YearOfBirth=""2006""><Coach FirstName=""Алла"" LastName=""Усенок""/><Coach FirstName=""Виталий"" LastName=""Барташевич""/><Team City=""Минск"" Name=""ГЦОР Трактор""/></Swimmer><Swimmer FirstName=""Максим"" LastName=""Кликушин"" YearOfBirth=""2007""><Coach FirstName=""Алла"" LastName=""Усенок""/><Coach FirstName=""Виталий"" LastName=""Барташевич""/><Team City=""Минск"" Name=""ГЦОР Трактор""/></Swimmer><Swimmer FirstName=""Матвей"" LastName=""Данкевич"" YearOfBirth=""2006""><Team City=""Молодечно"" Name=""СК Олимпик-2011""/></Swimmer><Swimmer FirstName=""Никита"" LastName=""Клюй"" YearOfBirth=""2005""><Team City=""Minsk"" Name=""ДЮСШ Янтарь""/></Swimmer>"


### For XML Raw

Cамый простой и быстрый способ генерации XML. 

Используется в случае если нет необходимости в нетривиальном формате создаваемого XML документа.


In [7]:
use tempdb
go

select  s.FirstName   swimmer_first_name,
		s.LastName    swimmer_last_name,
		s.YearOfBirth swimmer_year_of_birth,
		sc.City, 
		sc.[Name],
		c.FirstName   coach_first_name,
		c.LastName    coach_last_name
from dbo.Swimmer s
left join dbo.SwimmingClub sc   on sc.SwimmingClubID = s.SwimmingClubID
left join dbo.SwimmerCoach xref on xref.SwimmerID = s.SwimmerID 
inner join dbo.Coach c          on c.CoachID = xref.CoachID
order by 1, 2, 3, 4, 5, 6, 7
for xml raw



Commands completed successfully.

Total execution time: 00:00:00.0014396

(7 rows affected)

Total execution time: 00:00:00.0096049

XML_F52E2B61-18A1-11d1-B105-00805F49916B
"<row swimmer_first_name=""Gleb"" swimmer_last_name=""Bondarec"" swimmer_year_of_birth=""2007"" City=""Минск"" Name=""ГЦОР Трактор"" coach_first_name=""Алла"" coach_last_name=""Усенок""/><row swimmer_first_name=""Gleb"" swimmer_last_name=""Bondarec"" swimmer_year_of_birth=""2007"" City=""Минск"" Name=""ГЦОР Трактор"" coach_first_name=""Виталий"" coach_last_name=""Барташевич""/><row swimmer_first_name=""Анна"" swimmer_last_name=""Высоцкая"" swimmer_year_of_birth=""2007"" City=""Minsk"" Name=""ДЮСШ Янтарь"" coach_first_name=""Евгения"" coach_last_name=""Жукова""/><row swimmer_first_name=""Илья"" swimmer_last_name=""Гавриленко"" swimmer_year_of_birth=""2006"" City=""Минск"" Name=""ГЦОР Трактор"" coach_first_name=""Алла"" coach_last_name=""Усенок""/><row swimmer_first_name=""Илья"" swimmer_last_name=""Гавриленко"" swimmer_year_of_birth=""2006"" City=""Минск"" Name=""ГЦОР Трактор"" coach_first_name=""Виталий"" coach_last_name=""Барташевич""/><row swimmer_first_name=""Максим"" swimmer_last_name=""Кликушин"" swimmer_year_of_birth=""2007"" City=""Минск"" Name=""ГЦОР Трактор"" coach_first_name=""Алла"" coach_last_name=""Усенок""/><row swimmer_first_name=""Максим"" swimmer_last_name=""Кликушин"" swimmer_year_of_birth=""2007"" City=""Минск"" Name=""ГЦОР Трактор"" coach_first_name=""Виталий"" coach_last_name=""Барташевич""/>"


### For XML Auto

Чуть более сложный и одновременно чуть более мощный способ генерации XML. 

Позволяет стандартным (шаблонным) способом организовывать нетривиальную иерархическую структуру XML.


In [6]:
use tempdb
go

select  s.FirstName   swimmer_first_name,
		s.LastName    swimmer_last_name,
		s.YearOfBirth swimmer_year_of_birth,
		sc.City, 
		sc.[Name],
		c.FirstName   coach_first_name,
		c.LastName    coach_last_name
from dbo.Swimmer s
left join dbo.SwimmingClub sc   on sc.SwimmingClubID = s.SwimmingClubID
left join dbo.SwimmerCoach xref on xref.SwimmerID = s.SwimmerID 
inner join dbo.Coach c          on c.CoachID = xref.CoachID
order by 1, 2, 3, 4, 5, 6, 7
for xml auto



Commands completed successfully.

Total execution time: 00:00:00.0015640

(7 rows affected)

Total execution time: 00:00:00.0094259

XML_F52E2B61-18A1-11d1-B105-00805F49916B
"<s swimmer_first_name=""Gleb"" swimmer_last_name=""Bondarec"" swimmer_year_of_birth=""2007""><sc City=""Минск"" Name=""ГЦОР Трактор""><c coach_first_name=""Алла"" coach_last_name=""Усенок""/><c coach_first_name=""Виталий"" coach_last_name=""Барташевич""/></sc></s><s swimmer_first_name=""Анна"" swimmer_last_name=""Высоцкая"" swimmer_year_of_birth=""2007""><sc City=""Minsk"" Name=""ДЮСШ Янтарь""><c coach_first_name=""Евгения"" coach_last_name=""Жукова""/></sc></s><s swimmer_first_name=""Илья"" swimmer_last_name=""Гавриленко"" swimmer_year_of_birth=""2006""><sc City=""Минск"" Name=""ГЦОР Трактор""><c coach_first_name=""Алла"" coach_last_name=""Усенок""/><c coach_first_name=""Виталий"" coach_last_name=""Барташевич""/></sc></s><s swimmer_first_name=""Максим"" swimmer_last_name=""Кликушин"" swimmer_year_of_birth=""2007""><sc City=""Минск"" Name=""ГЦОР Трактор""><c coach_first_name=""Алла"" coach_last_name=""Усенок""/><c coach_first_name=""Виталий"" coach_last_name=""Барташевич""/></sc></s>"


### For XML Path

Самый гибкий способ генерации XML практически любой структуры (вместе с тем, и самый медленный!)

Тот же XML что и в примере про xml explicit

In [5]:
use tempdb
go

select  s.FirstName   [Swimmer/@FirstName],
		s.LastName    [Swimmer/@LastName],
		s.YearOfBirth [Swimmer/@YearOfBirth],
        sc.City       [Swimmer/Team/@City], 
        sc.[Name]     [Swimmer/Team/@Name],
		(
		   select
               c.FirstName   [@FirstName],
               c.LastName    [@LastName]
           from dbo.SwimmerCoach xref 
           inner join dbo.Coach c on c.CoachID = xref.CoachID
		   where xref.SwimmerID = s.SwimmerID 
		   for xml path('Coach'), type
		)             [Swimmer]
from dbo.Swimmer s
left join dbo.SwimmingClub sc   on sc.SwimmingClubID = s.SwimmingClubID
order by 1, 2, 3, 4, 5
for xml path('')



Commands completed successfully.

Total execution time: 00:00:00.0013479

(7 rows affected)

Total execution time: 00:00:00.0098099

XML_F52E2B61-18A1-11d1-B105-00805F49916B
"<Swimmer FirstName=""Gleb"" LastName=""Bondarec"" YearOfBirth=""2007""><Team City=""Минск"" Name=""ГЦОР Трактор""/><Coach FirstName=""Алла"" LastName=""Усенок""/><Coach FirstName=""Виталий"" LastName=""Барташевич""/></Swimmer><Swimmer FirstName=""Алексей"" LastName=""Рылько"" YearOfBirth=""2005""><Team City=""Minsk"" Name=""ДЮСШ Янтарь""/></Swimmer><Swimmer FirstName=""Анна"" LastName=""Высоцкая"" YearOfBirth=""2007""><Team City=""Minsk"" Name=""ДЮСШ Янтарь""/><Coach FirstName=""Евгения"" LastName=""Жукова""/></Swimmer><Swimmer FirstName=""Илья"" LastName=""Гавриленко"" YearOfBirth=""2006""><Team City=""Минск"" Name=""ГЦОР Трактор""/><Coach FirstName=""Алла"" LastName=""Усенок""/><Coach FirstName=""Виталий"" LastName=""Барташевич""/></Swimmer><Swimmer FirstName=""Максим"" LastName=""Кликушин"" YearOfBirth=""2007""><Team City=""Минск"" Name=""ГЦОР Трактор""/><Coach FirstName=""Алла"" LastName=""Усенок""/><Coach FirstName=""Виталий"" LastName=""Барташевич""/></Swimmer><Swimmer FirstName=""Матвей"" LastName=""Данкевич"" YearOfBirth=""2006""><Team City=""Молодечно"" Name=""СК Олимпик-2011""/></Swimmer><Swimmer FirstName=""Никита"" LastName=""Клюй"" YearOfBirth=""2005""><Team City=""Minsk"" Name=""ДЮСШ Янтарь""/></Swimmer>"


### Опции elements и root:

In [4]:
use tempdb
go

select  s.FirstName   ,
		s.LastName    ,
		s.YearOfBirth ,
        sc.City       Team_City, 
        sc.[Name]     Team_Name
from dbo.Swimmer s
left join dbo.SwimmingClub sc   on sc.SwimmingClubID = s.SwimmingClubID
order by 1, 2, 3, 4, 5
for xml raw('element'), root('persons'), elements




Commands completed successfully.

Total execution time: 00:00:00.0019112

(7 rows affected)

Total execution time: 00:00:00.0079248

XML_F52E2B61-18A1-11d1-B105-00805F49916B
<persons><element><FirstName>Gleb</FirstName><LastName>Bondarec</LastName><YearOfBirth>2007</YearOfBirth><Team_City>Минск</Team_City><Team_Name>ГЦОР Трактор</Team_Name></element><element><FirstName>Алексей</FirstName><LastName>Рылько</LastName><YearOfBirth>2005</YearOfBirth><Team_City>Minsk</Team_City><Team_Name>ДЮСШ Янтарь</Team_Name></element><element><FirstName>Анна</FirstName><LastName>Высоцкая</LastName><YearOfBirth>2007</YearOfBirth><Team_City>Minsk</Team_City><Team_Name>ДЮСШ Янтарь</Team_Name></element><element><FirstName>Илья</FirstName><LastName>Гавриленко</LastName><YearOfBirth>2006</YearOfBirth><Team_City>Минск</Team_City><Team_Name>ГЦОР Трактор</Team_Name></element><element><FirstName>Максим</FirstName><LastName>Кликушин</LastName><YearOfBirth>2007</YearOfBirth><Team_City>Минск</Team_City><Team_Name>ГЦОР Трактор</Team_Name></element><element><FirstName>Матвей</FirstName><LastName>Данкевич</LastName><YearOfBirth>2006</YearOfBirth><Team_City>Молодечно</Team_City><Team_Name>СК Олимпик-2011</Team_Name></element><element><FirstName>Никита</FirstName><LastName>Клюй</LastName><YearOfBirth>2005</YearOfBirth><Team_City>Minsk</Team_City><Team_Name>ДЮСШ Янтарь</Team_Name></element></persons>


# OpenXML
### Преобразование данных из XML в реляционный формат

In [9]:
declare @hdoc int,
	    	@xml  varchar(1000)

set @xml = '
<root>
  <Person City="Bothell" Address="New address">
    <Name>
      <FirstName>Syed</FirstName>
      <LastName>Abbas</LastName>
    </Name>
  </Person>
  <Person City="Carnation" Address="9752 Jeanne Circle">
    <Name>
      <FirstName>Kim</FirstName>
      <LastName>Abercrombie</LastName>
    </Name>
  </Person>
</root>  
'

--получаем дескриптор XML документа
exec sp_xml_preparedocument @hdoc OUTPUT, @xml

select * 
from 
OpenXML(@hdoc,'/root/Person',2)
	with(
			FirstName varchar(20) 'Name/FirstName',
			LastName varchar(20)  'Name/LastName',
			City varchar(100)     '@City'
		) X

--освобождаем ресурсы
exec sp_xml_removedocument @hdoc



(2 rows affected)

Total execution time: 00:00:00.0176054

FirstName,LastName,City
Syed,Abbas,Bothell
Kim,Abercrombie,Carnation


# For JSON Extention
### Преобразование данных из реляционной модели в JSON

Аналогично тому как мы можем генерировать XML, мы можем и с JSON:

In [17]:
use tempdb
go

select  s.FirstName   [Swimmer.FirstName],
		s.LastName    [Swimmer.LastName],
		s.YearOfBirth [Swimmer.YearOfBirth],
        sc.City       [Team.City], 
        sc.[Name]     [Team.Name],
		(
		   select
               c.FirstName   [FirstName],
               c.LastName    [LastName]
           from dbo.SwimmerCoach xref 
           inner join dbo.Coach c on c.CoachID = xref.CoachID
		   where xref.SwimmerID = s.SwimmerID 
		   for json path
		)             [Coach]
from dbo.Swimmer s
left join dbo.SwimmingClub sc   on sc.SwimmingClubID = s.SwimmingClubID
order by 1, 2, 3, 4, 5
for json path



Commands completed successfully.

Total execution time: 00:00:00.0019346

[{"Swimmer":{"FirstName":"Gleb","LastName":"Bondarec","YearOfBirth":2007},"Team":{"City":"Минск","Name":"ГЦОР Трактор"},"Coach":[{"FirstName":"Алла","LastName":"Усенок"},{"FirstName":"Виталий","LastName":"Барташевич"}]},{"Swimmer":{"FirstName":"Алексей","LastName":"Рылько","YearOfBirth":2005},"Team":{"City":"Minsk","Name":"ДЮСШ Янтарь"}},{"Swimmer":{"FirstName":"Анна","LastName":"Высоцкая","YearOfBirth":2007},"Team":{"City":"Minsk","Name":"ДЮСШ Янтарь"},"Coach":[{"FirstName":"Евгения","LastName":"Жукова"}]},{"Swimmer":{"FirstName":"Илья","LastName":"Гавриленко","YearOfBirth":2006},"Team":{"City":"Минск","Name":"ГЦОР Трактор"},"Coach":[{"FirstName":"Алла","LastName":"Усенок"},{"FirstName":"Виталий","LastName":"Барташевич"}]},{"Swimmer":{"FirstName":"Максим","LastName":"Кликушин","YearOfBirth":2007},"Team":{"City":"Минск","Name":"ГЦОР Трактор"},"Coach":[{"FirstName":"Алла","LastName":"Усенок"},{"FirstName":"Виталий","LastName":"Барташевич"}]},{"Swimmer":{"FirstName":"Матвей","LastName":"Данкевич","YearOfBirth":2006},"Team":{"City":"Молодечно","Name":"СК Олимпик-2011"}},{"Swimmer":{"FirstName":"Никита","LastName":"Клюй","YearOfBirth":2005},"Team":{"City":"Minsk","Name":"ДЮСШ Янтарь"}}]

Total execution time: 00:00:00.0079642

# OpenJSON
### Преобразование данных из JSON в реляционный формат


<b>OPENJSON</b> — функция с табличным значением, которая выполняет синтаксический анализ текста JSON и возвращает объекты и свойства из входных данных JSON в виде строк и столбцов. 

<a href='https://docs.microsoft.com/ru-RU/sql/t-sql/functions/openjson-transact-sql?view=sql-server-2017'>Link</a>

In [15]:
declare @json nvarchar(4000) 
set @json =	N'
{
    "FirstName": "Илья",
    "LastName": "Гавриленко",
    "YearOfBirth": 2006,
    "Gender": "м",
    "Club": {
        "Name": "ГЦОР Трактор",
        "City": "Минск"
    },
    "Category": "II",
    "Coach": [
        {"FirstName": "Алла", "LastName": "Усенок"},
        {"FirstName": "Виталий", "LastName": "Барташевич"}
    ]
} 
'

select *
from openjson(@json) 

--извлечь все атрибуты, в том числе, более чем 1 уровня вложенности
select *
from openjson(@json) 
	with 
		(
			FirstName       nvarchar(20)    N'$.FirstName',
			LastName        nvarchar(30)    N'$.LastName',
			YearOfBirth     smallint        N'$.YearOfBirth',
			Gender          nchar(1)        N'$.Gender',
			Category        nvarchar(20)    N'$.Category',
			ClubCity        nvarchar(50)    N'$.Club.City',
			ClubName        nvarchar(100)   N'$.Club.Name',
			Coach           nvarchar(max)   N'$.Coach'        as json
		)  js 




(7 rows affected)

(1 row affected)

Total execution time: 00:00:00.0173023

key,value,type
FirstName,Илья,1
LastName,Гавриленко,1
YearOfBirth,2006,2
Gender,м,1
Club,"{ ""Name"": ""ГЦОР Трактор"", ""City"": ""Минск"" }",5
Category,II,1
Coach,"[ {""FirstName"": ""Алла"", ""LastName"": ""Усенок""}, {""FirstName"": ""Виталий"", ""LastName"": ""Барташевич""} ]",4


FirstName,LastName,YearOfBirth,Gender,Category,ClubCity,ClubName,Coach
Илья,Гавриленко,2006,м,II,Минск,ГЦОР Трактор,"[ {""FirstName"": ""Алла"", ""LastName"": ""Усенок""}, {""FirstName"": ""Виталий"", ""LastName"": ""Барташевич""} ]"


Если вы хотите получить данные из вложенного массива JSON, можно использовать корреляционный вызов openjson используя ```CROSS APPLY``` или ```OUTER APPLY```:

In [19]:
declare @json nvarchar(4000) 
set @json =	N'
{
    "FirstName": "Илья",
    "LastName": "Гавриленко",
    "YearOfBirth": 2006,
    "Gender": "м",
    "Club": {
        "Name": "ГЦОР Трактор",
        "City": "Минск"
    },
    "Category": "II",
    "Coach": [
        {"FirstName": "Алла", "LastName": "Усенок"},
        {"FirstName": "Виталий", "LastName": "Барташевич"}
    ]
} 
'

--извлечь все атрибуты, в том числе, более чем 1 уровня вложенности
select 		js.FirstName,
			js.LastName,
			js.YearOfBirth,
			js.Gender,
			js.Category,
			js.ClubCity,
			js.ClubName,
			jsc.CoachFirstName, 
			jsc.CoachLastName
from openjson(@json) 
	with 
		(
			FirstName       nvarchar(20)    N'$.FirstName',
			LastName        nvarchar(30)    N'$.LastName',
			YearOfBirth     smallint        N'$.YearOfBirth',
			Gender          nchar(1)        N'$.Gender',
			Category        nvarchar(20)    N'$.Category',
			ClubCity        nvarchar(50)    N'$.Club.City',
			ClubName        nvarchar(100)   N'$.Club.Name',
			Coach           nvarchar(max)   N'$.Coach'        as json
		)  js 
outer apply openjson(js.Coach)        
	with 
		(
			CoachFirstName  nvarchar(20)    N'$.FirstName',
			CoachLastName   nvarchar(30)    N'$.LastName'
		)  jsc 



(2 rows affected)

Total execution time: 00:00:00.0071696

FirstName,LastName,YearOfBirth,Gender,Category,ClubCity,ClubName,CoachFirstName,CoachLastName
Илья,Гавриленко,2006,м,II,Минск,ГЦОР Трактор,Алла,Усенок
Илья,Гавриленко,2006,м,II,Минск,ГЦОР Трактор,Виталий,Барташевич


### JSON_Value, JSON_Query, JSON_Modify

In [3]:
declare @json nvarchar(4000) 
set @json =	N'
{
    "FirstName": "Илья",
    "LastName": "Гавриленко",
    "YearOfBirth": 2006,
    "Gender": "м",
    "Club": {
        "Name": "ГЦОР Трактор",
        "City": "Минск"
    },
    "Category": "II",
    "Coach": [
        {"FirstName": "Алла", "LastName": "Усенок"},
        {"FirstName": "Виталий", "LastName": "Барташевич"}
    ]
} 
'

select FirstName   = json_value(@json, '$.FirstName'),
       LastName    = json_value(@json, '$.LastName'),
       YearOfBirth = json_value(@json, '$.YearOfBirth'),
       Team        = json_value(@json, '$.Club.City') + ' ' + json_value(@json, '$.Club.Name'),
       YearOfBirth = json_query(@json, '$.Coach')   --json_value не подходит



(1 row affected)

Total execution time: 00:00:00.0046983

FirstName,LastName,YearOfBirth,Team,YearOfBirth
Илья,Гавриленко,2006,Минск ГЦОР Трактор,"[ {""FirstName"": ""Алла"", ""LastName"": ""Усенок""}, {""FirstName"": ""Виталий"", ""LastName"": ""Барташевич""} ]"


In [18]:
declare @json nvarchar(4000) 
set @json =	N'
{
    "FirstName": "Илья",
    "LastName": "Гавриленко",
    "YearOfBirth": 2006,
    "Gender": "м",
    "Club": {
        "Name": "ГЦОР Трактор",
        "City": "Минск"
    },
    "Category": "II",
    "Coach": [
        {"FirstName": "Алла", "LastName": "Усенок"},
        {"FirstName": "Виталий", "LastName": "Барашевич"}
    ]
} 
'

--корректируем ошибку в фамилии тренера
set @json = json_modify(@json, '$.Coach[1].LastName', N'Барташевич')   

--добавляем дополнительный атрибут
set @json = json_modify(@json, '$.Club.Address', N'Долгобродская улица, 37. ст.м. Тракторный Завод')   

--добавляем дополнительный элемент в массив
set @json = json_modify(
                @json, 
                'append $.Coach', 
                json_query(N'{"FirstName": "Наталья", "LastName": "Бученкова"}', '$')
            )   

select @json


(1 row affected)

Total execution time: 00:00:00.0081801

(No column name)
"{ ""FirstName"": ""Илья"", ""LastName"": ""Гавриленко"", ""YearOfBirth"": 2006, ""Gender"": ""м"", ""Club"": { ""Name"": ""ГЦОР Трактор"", ""City"": ""Минск"" ,""Address"":""Долгобродская улица, 37. ст.м. Тракторный Завод""}, ""Category"": ""II"", ""Coach"": [ {""FirstName"": ""Алла"", ""LastName"": ""Усенок""}, {""FirstName"": ""Виталий"", ""LastName"": ""Барташевич""} ,{""FirstName"": ""Наталья"", ""LastName"": ""Бученкова""}] }"
